In [528]:
import pandas as pd
import numpy as np
from ipywidgets import IntProgress
from matplotlib import pyplot as plt

In [529]:
dif = 9.
cur = 0
dig = 1
Fs = [{'label':'f1', 'F':1}]
for i in range(0, 10):
    dif = dif/10
    cur = round(cur+ dif, dig)
    dig +=1
    Fs.append({'label':'f'+str(dig), 'F':cur})

In [539]:
Fs

[{'label': 'f1', 'F': 1},
 {'label': 'f2', 'F': 0.9},
 {'label': 'f3', 'F': 0.99},
 {'label': 'f4', 'F': 0.999},
 {'label': 'f5', 'F': 0.9999},
 {'label': 'f6', 'F': 0.99999},
 {'label': 'f7', 'F': 0.999999},
 {'label': 'f8', 'F': 0.9999999},
 {'label': 'f9', 'F': 0.99999999},
 {'label': 'f10', 'F': 0.999999999},
 {'label': 'f11', 'F': 0.9999999999}]

In [530]:
chunkMapDF = pd.read_csv('data/ChunkMap.csv')
# qrVals = pd.read_csv('QR_Values.csv')

In [531]:
def leastSquare(X, Y, F):
    H_TRANSPOSE = np.vstack((np.linspace(1,1,len(X)), X))
    H = np.transpose(H_TRANSPOSE)
    currF = 1.
    for i in range(0, len(H_TRANSPOSE[0])):
        v = len(H)-i-1
#         print(v)
        currF = currF*F
        H_TRANSPOSE[0][v], H_TRANSPOSE[1][v] = H_TRANSPOSE[0][v]*currF, H_TRANSPOSE[1][v]*currF
    return np.linalg.inv(H_TRANSPOSE @ H) @ H_TRANSPOSE @ Y

def interpXY(X, Y):
    numPoints = pd.Series(X).max()+1
    startX = X[0]
    newX = list(range(startX, numPoints+startX))
    newY = [np.nan]*numPoints    
    for i in range(0, len(X)):
        xV = X[i]-startX
#         print(i, xV)
        newY[xV] = Y[i]
    newY = list(pd.Series(newY).interpolate())
    return newX, newY

def raiseAB(A, B):
    return np.exp(A), B

In [532]:
def genABData(A, B, Xf):
    X = list(range(0, Xf))
    Y = []
    for x in X:
        Y.append(A*np.exp(B*x))
    return X, Y

In [533]:
def processRegr(X, Y, F):
    logY = list(np.log(Y))
    nX, nY = interpXY(X, logY)
    A, B = leastSquare(nX, nY, F)
    return raiseAB(A, B)
# processRegr(X, Y, .99)

In [534]:
# plt.plot(X, Y)
# A, B = processRegr(X, Y, .99999)
# pred = genABData(A, B, pd.Series(X).max())
# plt.plot(pred[0], pred[1])

In [535]:
df = pd.read_csv('data/Chunks_Scaled/chunk1.csv')

In [536]:
df.head()

,Unnamed: 0,ContestId,EntriesScaled,TimeScaled,Before4HoursOut
0,0,55826033,0.000005,0.000000,True
1,1,55826033,0.000019,0.000028,True
2,2,55826033,0.000024,0.000128,True
3,3,55826033,0.000029,0.000199,True
4,4,55826033,0.000034,0.000228,True


In [538]:
cSeriesDF

,ContestId,MinutesRemaining,Entries
150963,58797657,1,18


In [537]:
chunkNames = chunkMapDF['Chunk'].unique()[:]

chunkBar = IntProgress(min=0, max=len(chunkNames))
contestBar = IntProgress(min=0, max=10000)

print("Chunks:")
display(chunkBar)

print("Contests:")
display(contestBar)

results = []

for chunkName in chunkNames:
    print(chunkName)
    chunkDF = pd.read_csv('data/Chunks_Scaled/'+chunkName+'.csv').drop(columns=['Unnamed: 0']).sort_values(by=['MinutesRemaining'], ascending=False)
    
    contests = chunkDF['ContestId'].unique()[:]
    contestBar.value=0
    contestBar.max = len(contests)
    for cid in contests:
        cSeriesDF = chunkDF[chunkDF['ContestId']==cid]
#         numberToRemove = len(cSeriesDF[cSeriesDF['MinutesRemaining']<240])

#         X = cSeriesDF['MinutesRemaining'].max() - cSeriesDF['MinutesRemaining'][:-numberToRemove]
#         Y = cSeriesDF['Entries'].cumsum()[:-numberToRemove]
#         X, Y = list(X), list(Y)        
#         count = 0
#         contest = {'ContestId':cid}
        
#         for fFactor in Fs:
            
#             fName = fFactor['label']
#             fFactor = fFactor['F']
# #             print(fName, fFactor)
#             A, B = 'TooShort', 'TooShort'
#             if(len(X)>=3):
#                 A, B = processRegr(X, Y, fFactor)
#             contest['A'+fName], contest['B'+fName]=A, B    
#         results.append(contest)
#         contestBar.value+=1
#     chunkBar.value+=1


Chunks:


IntProgress(value=0, max=65)

Contests:


IntProgress(value=0, max=10000)

chunk1


KeyError: 'MinutesRemaining'

In [526]:
resultsDF = pd.DataFrame(results).set_index('ContestId')

In [527]:
resultsDF.head()

,f10A,f10B,f11A,f11B,f1A,f1B,f2A,f2B,f3A,f3B,...,f5A,f5B,f6A,f6B,f7A,f7B,f8A,f8B,f9A,f9B
ContestId,,,,,,,,,,,,,,,,,,,,,
55826033,402.019,6.38683e-05,401.848,6.38737e-05,401.829,6.38743e-05,inf,-5.38728,inf,-0.482865,...,2.0541e+99,-0.0032119,266476,-2.67541e-05,655.437,5.7551e-05,421.393,6.32663e-05,403.738,6.38137e-05
56610037,186.177,8.7691e-05,186.146,8.76957e-05,186.143,8.76962e-05,inf,-3.90998,inf,-0.347349,...,1.09651e+28,-0.00181771,1585.99,2.53652e-05,224.648,8.24326e-05,189.629,8.7179e-05,186.488,8.76445e-05
56751298,0.661184,0.000231577,0.661178,0.000231577,0.661178,0.000231577,inf,-2.39249,inf,-0.21572,...,399.102,-2.11577e-06,0.745566,0.000235708,0.667472,0.000232144,0.661789,0.000231635,0.661239,0.000231582
56859642,23.552,0.000134688,23.5503,0.000134691,23.5501,0.000134691,inf,-2.85438,inf,-0.251052,...,8.9297e+09,-0.000829168,56.8338,9.66528e-05,25.524,0.000131323,23.7387,0.000134359,23.5689,0.000134658
57008091,7.00538,0.00016179,7.00511,0.000161792,7.00508,0.000161792,inf,-2.19106,inf,-0.191655,...,81928.9,-0.000405195,11.1534,0.000137387,7.31382,0.000159598,7.03513,0.000161575,7.00808,0.000161771


In [525]:
resultsDF.to_csv('data/LR_Values.csv')